In [13]:
# Import modules
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets

The file needs to be a CSV
Date/Time columns needs to be called 'Date Time'
Upto 4 data columns can be used and must be named 'LAeq', 'LAFmax', 'LA90', 'LA10'

In [14]:
fpath=input("File path: ")
fname=input("File name including extension: ")

File path: C:\Data\Python\Graphing
File name including extension: data.csv


In [15]:
#Set colours
ds_darkgreen='#008264'
ds_darkgreen_tint='#339b83'
ds_lightgreen='#6eaa6e'
ds_lightgreen_tint='#8bbb8b'
ds_blue='#0098a9'
ds_blue_tint='#33adba'

In [16]:
# Load data and calculate date range
df=pd.read_csv(fpath+"\\"+fname)
df['Date Time']=pd.to_datetime(df['Date Time'],format='%d/%m/%Y %H:%M')
d_min=pd.to_datetime(df['Date Time'].min().strftime('%d/%m/%Y'))
d_max=pd.to_datetime(df['Date Time'].max().strftime('%d/%m/%Y'))
rng=pd.date_range(d_min-pd.Timedelta(1,unit='d'),d_max,freq='D')
rng0=rng+pd.Timedelta(23,unit='H')
rng1=rng+pd.Timedelta(31,unit='H')
if rng1[0]<pd.to_datetime(df['Date Time'].min().strftime('%d/%m/%Y %H:%M')):
    rng0=rng0[1:]
    rng1=rng1[1:]
if rng0[-1]>pd.to_datetime(df['Date Time'].max().strftime('%d/%m/%Y %H:%M')):
    rng0=rng0[:-1]
    rng1=rng1[:-1]

In [17]:
# Create Plot
fig=go.Figure()

In [18]:
# Add traces
if 'LA10' in df.columns:
    fig.add_trace(go.Scatter(x=df['Date Time'],y=df['LA10'],name='$L_{A10}$',line=dict(color=ds_blue_tint,width=1)))
if 'LA90' in df.columns:
    fig.add_trace(go.Scatter(x=df['Date Time'],y=df['LA90'],name='$L_{A90}$',line=dict(color=ds_blue,width=1)))
if 'LAFmax' in df.columns:
    fig.add_trace(go.Scatter(x=df['Date Time'],y=df['LAFmax'],name='$L_{AFmax}$',line=dict(color=ds_lightgreen,width=1)))
if 'LAeq' in df.columns:
    fig.add_trace(go.Scatter(x=df['Date Time'],y=df['LAeq'],name='$L_{Aeq}$',line=dict(color=ds_darkgreen,width=1)))

In [19]:
# Add night indicators
for i in range(len(rng0)):
    fig.add_vrect(x0=rng0[i],x1=rng1[i],fillcolor=ds_blue,opacity=0.25,line_width=0,annotation_text="Night",annotation_font_family="Arial",annotation_position="top left")

In [20]:
hours=6
minhr=int(df['Date Time'].min().strftime('%H'))
maxhr=int(df['Date Time'].max().strftime('%H'))
s_range=pd.to_datetime(df['Date Time'].min().strftime('%d/%m/%Y'))+pd.Timedelta(minhr-(minhr%6),unit='H')
#s_range=pd.to_datetime(df['Date Time'].min().strftime('%d/%m/%Y %H'))-pd.Timedelta(1,unit='H')
#e_range=pd.to_datetime(df['Date Time'].max().strftime('%d/%m/%Y %H'))+pd.Timedelta(1,unit='H')
e_range=pd.to_datetime(df['Date Time'].max().strftime('%d/%m/%Y'))+pd.Timedelta(maxhr+(maxhr%6),unit='H')

In [21]:
fig.update_layout(
    plot_bgcolor='white',
    title="Measured Data Time History",
    width=1500,
    height=800,
    yaxis=dict(
        title='<b>dB<b>',
        dtick=5,
        gridcolor='#dedfe0',
    ),
    xaxis=dict(
        title='<b>Date and Time<b>',
        gridcolor='#dedfe0',
        range=[s_range,e_range],
        tick0=s_range,
        dtick=1000*60*60*hours,
        tickformat="%H:%M\n%d %b<br>%Y"
    ),
    font_family="Arial",
    legend=dict(
        orientation="v",
        yanchor="middle",
        valign='middle',
        y=0.5,
        xanchor="left",
        x=1.01,
        title=dict(
            text="Key\n",
            font_size=14,
            side='top left'
        ),
        tracegroupgap=20,
        font_size=12,
        bordercolor="Black",
        borderwidth=0.75
    )
)
fig.show()

In [23]:
file_type=widgets.RadioButtons(
    options=['SVG', 'PNG', 'JPG','PDF'],
    description='Export Format:',
    disabled=False,
)
button = widgets.Button(
    description='Export Image',
    disabled=False,
    button_style='',
    tooltip=f'Export Image to {fpath}',
    icon='check' 
)
def on_button_clicked(b):
    fig.write_image(f"{fpath}\\{fname[:-4]}.{file_type.value.lower()}")
button.on_click(on_button_clicked)

display(file_type)
display(button)

RadioButtons(description='Export Format:', options=('SVG', 'PNG', 'JPG', 'PDF'), value='SVG')

Button(description='Export Image', icon='check', style=ButtonStyle(), tooltip='Export Image to C:\\Data\\Pytho…